Biblioteki:

In [ ]:
import os
from PIL import Image
from skimage.feature import graycomatrix, graycoprops
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import train_test_split


1. Wczytywanie obrazów oraz wycinanie z nich fragmentów (czyli próbek tekstury) o zadanym rozmiarze (np. 128 x 128). Wycięte fragmenty zostały zapisane do odpowiednich katalogów.

In [ ]:
initial_photos = 'Images'  # name of the directory containing the input photos
texture_samples = 'T_images'  # the name of the directory where the iter_texture samples will be saved
new_size_tSample = 128  # the size of each iter_texture S (128x128 pixels)

for name_of_image in os.listdir(initial_photos):
    # extracting the filename without the file extension from the name_of_image variable
    image_without_extension = name_of_image.split(".")[0]
    new_image_path = os.path.join(initial_photos, name_of_image)  # create new_path to image
    new_text_path = os.path.join(texture_samples, image_without_extension)  # create new_path to iter_texture directory

    try:  # try to create directory for Text_samples
        os.mkdir(new_text_path)
    except FileExistsError:
        print('Error: Directory for T_images already exists')

    open_image = Image.open(new_image_path)  # open image; assigns the opened image object to the open_image
    # line calculates the number of iter_texture cuts in the x-direction and y-direction; divides the width of the open_image by the
    # new_size_tSample, which represents the desired size of the iter_texture samples
    number_of_xdirection_cuts = round(open_image.size[0] / new_size_tSample) - 1  # result is rounded to the nearest integer and assigned to the variable
    number_of_ydirection_cuts = round(open_image.size[1] / new_size_tSample) - 1

    for x in range(
            number_of_xdirection_cuts):  # set up nested loops to iterate over the x and y coordinates of the iter_texture samples
        for y in range(number_of_ydirection_cuts):
            # creates a unique name for each iter_texture S by combining image_without_extension, the current
            # x index, the underscore character (_), the current y index, and the ".jpg" file extension
            name_for_tSample = image_without_extension + str(x) + '_' + str(y) + '.jpg'
            # crops a section of the open_image using the coordinates calculated based on the current x and y indices
            # crop() - specifying the coordinates of the top-left and bottom-right corners of the desired section
            cropped_image_by_using_coordinates = open_image.crop((x * new_size_tSample, y * new_size_tSample, (x + 1) * new_size_tSample, (y + 1) * new_size_tSample))
            # creates the new_path where the iter_texture S will be saved by combining the texture_samples directory new_path, image_without_extension
            # and the name_for_tSample
            new_path = os.path.join(texture_samples, image_without_extension, name_for_tSample)
            # saves the cropped iter_texture S as a new JPEG file at the specified new_path
            cropped_image_by_using_coordinates.save(new_path)

            # (x * new_size_tSample, y * new_size_tSample) -> top-left corner of the cropping region
            # (x + 1) and (y + 1) are used to calculate the bottom-right corner of the cropping region by obtaining the next x and y coordinates

Error: Directory for T_images already exists
Error: Directory for T_images already exists
Error: Directory for T_images already exists


2. Wczytywanie próbek tekstury i wyznaczanie dla nich cech tekstury na
podstawie modelu macierzy zdarzeń (ang. grey-level co-occurrence matrix) za pomocą odpowiednich funkcji z pakietu scikit-image. Przed obliczeniem cech, każdy obraz został przekształcony do skali szarości oraz została zmnniejszona głębia jasności do 5 bitów (64 poziomy).

In [ ]:
texture_samples = 'T_images'  # stores the name of the folder containing iter_texture samples
distances_between_pixels = [1, 3,
                            5]  # a list of distances between pixels used for the grey-level co-occurrence matrix (GLCM)
list_of_angles = [0, np.pi / 4, np.pi / 2,
                  (3 / 4) * np.pi]  # a list of list_of_angles used for the GLCM -> [0, 45, 90, 135]

# create a dictionary res_cat to store the results specific to that category
# iterating over the folders within the texture_samples directory using os.listdir(texture_samples)

data_df = pd.DataFrame()  # create a DataFrame from the data list

for Ftexture in os.listdir(texture_samples):
    # constructs the path to the current iter_texture folder by joining the texture_samples directory path and the current folder name
    path_for_texture_sampling = os.path.join(texture_samples, Ftexture)

    for Stext in os.listdir(path_for_texture_sampling):
        # constructs the path to the current iter_texture S file by joining the path_for_texture_sampling and the current file name (Stext)
        Spath = os.path.join(path_for_texture_sampling, Stext)
        # opens the image file using the Image module from the PIL library; then converts the image to grayscale ('L' mode) using the convert method
        S = Image.open(Spath).convert('L')
        # reduces the brightness depth of the grayscale image to 5 bits by dividing each pixel value by 4 using a lambda function and the point method
        Sgray_reduced = S.point(lambda x: int(x / 4))
        # converts the reduced grayscale image (Sgray_reduced) to a NumPy array (Sgray_array)
        Sgray_array = np.array(Sgray_reduced)

        # calculates the GLCM (grey-level co-occurrence matrix) using the pixel distances, list_of_angles, and levels specified
        # Sgray_array: The reduced grayscale image array
        # distances: A list of distances between pixels used for the GLCM
        # list_of_angles: A list of list_of_angles at which to compute the GLCM
        # levels: The number of gray levels to use when computing the GLCM
        # symmetric: A boolean flag indicating whether the GLCM should be symmetric (True) or not (False)
        GLCM_calculate = graycomatrix(Sgray_array, distances=distances_between_pixels, angles=list_of_angles, levels=64,
                                      symmetric=True)

        res_cat = pd.Series({'category': Ftexture,
                             'dissimilarity': graycoprops(GLCM_calculate, 'dissimilarity'),
                             'correlation': graycoprops(GLCM_calculate, 'correlation'),
                             'contrast': graycoprops(GLCM_calculate, 'contrast'),
                             'energy': graycoprops(GLCM_calculate, 'energy'),
                             'homogeneity': graycoprops(GLCM_calculate, 'homogeneity'),
                             'ASM': graycoprops(GLCM_calculate, 'ASM')
                             })

        data_df = pd.concat([data_df, res_cat.to_frame().T])  # merge arrays

data_df.to_pickle('results.pkl')  # save results to .pkl file

print('Thats all')

Thats all


3.  Skrypt służący do klasyfikacji wektorów cech z wykorzystaniem dowolnego algorytmu klasyfikacji danych dostępnego w pakiecie scikit-learn. Zostało obliczona i wyświetlona na ekranie wyznaczona dokładność klasyfikatora.

In [ ]:
# Read features from the .pkl file
features_from_pkl_file = pd.read_pickle('results.pkl')

# Define the properties to extract from the file
properties_from_file = ['dissimilarity', 'correlation', 'contrast', 'energy', 'homogeneity', 'ASM']

# Extract unique iter_texture categories
T_images = features_from_pkl_file['category'].unique()

# Create empty lists for features and labels
features = []
labels = []

# Iterate over each iter_texture category
for iter_texture in T_images:
    # Filter the data for the current iter_texture category
    texture_data = features_from_pkl_file[features_from_pkl_file['category'] == iter_texture]

    # Extract the properties for the current iter_texture category
    properties = texture_data[properties_from_file].values

    # Flatten and add the properties to the features list
    features.extend([np.hstack(prop).flatten() for prop in properties])

    # Add labels for each sample in the current iter_texture category
    labels.extend([iter_texture] * len(properties))

# Split the dataset into training set and test set
x_train, x_test, y_train, y_test = train_test_split(features, labels)

# Create an SVM classifier
classifier = svm.SVC(kernel='linear')

# Train the model using the training sets
classifier.fit(x_train, y_train)

# Predict the response for the test dataset
predict_response = classifier.predict(x_test)

# Calculate the model accuracy
accuracy = metrics.accuracy_score(y_test, predict_response) * 100
print('Accuracy:', round(accuracy, 2), '%')


Accuracy: 100.0 %
